# Fine-Tuning Amazon Titan Image Generator G1

> ☝️ This notebook has been tested with the **`SageMaker Data Science 3.0`** kernel in Amazon SageMaker Studio.

---

In this notebook, we will show how to fine tune [Amazon Titan Image Generator G1](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-image-models.html) on [Amazon Bedrock](https://aws.amazon.com/bedrock/) model.

We will teach our model to recognize two new classes:

**Ron the dog**

<img src="data/ron_01.jpg" width="25%" height="25%" style="float: left"/>
<img src="data/ron_06.jpg" width="25%" height="25%" style="float: left" />
<img src="data/ron_13.jpg" width="25%" height="25%" style="float: left" />
<img src="data/ron_20.jpg" width="25%" height="25%" style="float: left" />

and  **Smila the cat**

<img src="data/smila_02.jpg" width="25%" height="25%" style="float: left"/>
<img src="data/smila_06.jpg" width="25%" height="25%" style="float: left" />
<img src="data/smila_15.jpg" width="25%" height="25%" style="float: left" />
<img src="data/smila_24.jpg" width="25%" height="25%" style="float: left" />

In [2]:
!pip install --upgrade --force-reinstall --no-cache boto3
!pip install --upgrade --force-reinstall --no-cache botocore
!pip install --upgrade --force-reinstall --no-cache awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 360.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 339.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 315.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 391.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 357.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Uninstalling urllib3-2.2.2:
      Successfully uninstalled urllib3-2.2.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-date

## Pre-requisites

Import needed libraries and instantiate the needed clients

In [3]:
#Libraries
import json
import boto3
import datetime
import time

# Boto3 clients
s3_client = boto3.client('s3')
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
bedrock_client = boto3.client('bedrock')
bedrock_runtime_client = boto3.client('bedrock-runtime')
# Account and region info
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]


### Create an Amazon S3 bucket
Create a bucket where your training data will be stored

In [4]:
bucket_name = "amazonbedrockft-imagegen-ronsmila-{}-{}".format(account_id, region)
role_name = "AmazonBedrockFineTuning-imagegen-ronsmila"
s3_bedrock_ft_access_policy="AmazonBedrockFT-ImageGen-S3-ronsmila"
customization_role = f"arn:aws:iam::{account_id}:role/{role_name}"

try:
    if region != 'us-east-1':
        s3_client.create_bucket(
            Bucket=bucket_name,     
            CreateBucketConfiguration={
                'LocationConstraint': region
            },
        )
    else:
        s3_client.create_bucket(Bucket=bucket_name)
    print("AWS Bucket: {}".format(bucket_name))
except Exception as err:
    print("ERROR: {}".format(err))

s3_bucket_path = "s3://{}".format(bucket_name)
print("S3 bucket path: {}".format(s3_bucket_path))

AWS Bucket: amazonbedrockft-imagegen-ronsmila-094784590684-us-east-1
S3 bucket path: s3://amazonbedrockft-imagegen-ronsmila-094784590684-us-east-1


## Data preparation
- To fine-tune a text-to-image or image-to-embedding model, prepare a training dataset by create a JSONL file with multiple JSON lines. 
- Validation datasets are not supported. 
- Each JSON line is a sample containing an image-ref, the Amazon S3 URI for an image, and a caption that could be a prompt for the image.

The images must be in JPEG or PNG format.

    {"image-ref": "s3://bucket/path/to/image001.png", "caption": "<prompt text>"}
    {"image-ref": "s3://bucket/path/to/image002.png", "caption": "<prompt text>"}
    {"image-ref": "s3://bucket/path/to/image003.png", "caption": "<prompt text>"}    

The following is an example item:

    {"image-ref": "s3://my-bucket/my-pets/cat.png", "caption": "an orange cat with white spots"}

#### Locate your sample json file
We are going to use a json file which contains the image captions in the following format:

    {
        "imagefile":"caption",
        "imagefile":"caption",
        "imagefile":"caption"
    }

In [5]:
raw_data_file = "prompts/captions.json"

with open(raw_data_file, 'r') as file:
    raw_data = json.load(file)

print(raw_data)

{'ron_01.jpg': 'Ron the dog laying on a white dog bed', 'ron_02.jpg': 'Ron the dog sitting on a tile floor, possibly in a kitchen or living room', 'ron_03.jpg': 'Ron the dog laying on a car seat', 'ron_04.jpg': 'Ron the dog looking directly at the camera. He is laying down on a wooden floor.', 'ron_05.jpg': 'Ron the dog sitting on a couch, looking at the camera with a smile on his face.', 'ron_06.jpg': 'Ron the dog lying on a couch, covered in a blanket.', 'ron_07.jpg': 'Ron the dog sleeping or resting, with his head on the stuffed animal.', 'ron_08.jpg': 'Ron the dog sitting on a box with a red leash.', 'ron_09.jpg': 'Ron the dog sitting in the snow, wearing a red collar.', 'ron_10.jpg': 'Ron the dog lying on a couch, chewing on a tennis shoe.', 'ron_11.jpg': 'Ron the dog sitting on a sandy beach, wearing a blue collar.', 'ron_12.jpg': 'Ron the dog wearing a yellow raincoat and is sitting on the floor.', 'ron_13.jpg': 'Ron the dog looking at the camera in front of a table.', 'ron_14.j

### Create the dataset file and upload the images to Amazon S3
Create the `jsonl` file with the images prompt based on the image's s3 path. 

In [6]:
images_dir = 'data'
output_file = 'prompts/output.jsonl'


with open(output_file, "w", encoding="utf-8") as jsonl_file:
    for filename, caption in raw_data.items():
        image_path = "{}/{}".format(images_dir, filename)
        s3_image_path = "{}/{}".format(s3_bucket_path, image_path)
        jsonl_entry = {
            "image-ref":s3_image_path,
            "caption": caption
        }
        jsonl_file.write(json.dumps(jsonl_entry, ensure_ascii=False) + "\n")
        s3_client.upload_file(image_path, bucket_name, image_path)
    # Remove the newline character from the last line
    jsonl_file.seek(jsonl_file.tell() - 1)
    jsonl_file.truncate()

s3_client.upload_file(output_file, bucket_name, output_file)

## Fine tune job preparation - Creating role and policies requirements

We will now prepare the necessary role for the fine-tune job. That includes creating the policies required to run customization jobs with Amazon Bedrock.

### Create Trust relationship
This JSON object defines the trust relationship that allows the bedrock service to assume a role that will give it the ability to talk to other required AWS services. The conditions set restrict the assumption of the role to a specfic account ID and a specific component of the bedrock service (model_customization_jobs)

In [7]:
# This JSON object defines the trust relationship that allows the bedrock service to assume a role that will give it the ability to talk to other required AWS services. The conditions set restrict the assumption of the role to a specfic account ID and a specific component of the bedrock service (model_customization_jobs)
ROLE_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Principal": {{
                "Service": "bedrock.amazonaws.com"
            }},
            "Action": "sts:AssumeRole",
            "Condition": {{
                "StringEquals": {{
                    "aws:SourceAccount": "{account_id}"
                }},
                "ArnEquals": {{
                    "aws:SourceArn": "arn:aws:bedrock:{region}:{account_id}:model-customization-job/*"
                }}
            }}
        }}
    ]
}}
"""

### Create S3 access policy

This JSON object defines the permissions of the role we want bedrock to assume to allow access to the S3 bucket that we created that will hold our fine-tuning datasets and allow certain bucket and object manipulations.


In [8]:
ACCESS_POLICY_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:DeleteObject",
                "s3:PutObject",
                "s3:GetObject",
                "s3:GetBucketAcl",
                "s3:GetBucketNotification",
                "s3:ListBucket",
                "s3:PutBucketNotification"
            ],
            "Resource": [
                "arn:aws:s3:::{bucket_name}",
                "arn:aws:s3:::{bucket_name}/*"
            ]
        }}
    ]
}}"""

### Create IAM role and attach policies

Let's now create the IAM role with the created trust policy and attach the s3 policy to it

In [9]:
response = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=ROLE_DOC,
    Description="Role for Bedrock to access S3 for finetuning",
)

In [10]:
role_arn = response["Role"]["Arn"]
response = iam_client.create_policy(
    PolicyName=s3_bedrock_ft_access_policy,
    PolicyDocument=ACCESS_POLICY_DOC,
)
policy_arn = response["Policy"]["Arn"]
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn,
)

{'ResponseMetadata': {'RequestId': '400168ac-7170-4f64-8231-820ff457d18c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 13:43:02 GMT',
   'x-amzn-requestid': '400168ac-7170-4f64-8231-820ff457d18c',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

## Create Fine-tuning job

<div class="alert alert-block alert-info">
    <b>Note:</b> Fine-tuning job will take around 5 hours to complete to complete with 60 images and 8000 steps. Amazon Titan Image Generator G1 fine-tuning pricing model is based on the price per images seen. The number of images seen depends on the batch size, number of steps and images provided. The cost for fine-tuning the Titan image Generator G1 is defined as: <code>(min(count of images provided, batch size) * (number of steps) * (cost per image seen)) + (monthly cost to storage the model)</code>. For instance, the cost for training with the 60 images, 8000 steps and batch size of 8 will be <code>(min(60, 8)* 8000 * (cost per image seen)) + (monthly cost to storage the model)</code> In the us-east-1 region as of Feb 2024, this would be <code>(8 * 8000 * 0.005)(per customization job) + 1.95(per_month) = 320 dollars for customize the model + 1.95 per month to store the model</code> More information on the pricing model can be found <a ref="https://aws.amazon.com/bedrock/pricing/">here</a>
</div>

Now that we have all the requirements in place, let's create the fine-tuning job with the Titan Image Generator model.

To do so, we need to set the model **hyperparameters** for `stepCount`, `batchSize` and `learningRate` and provide the path to your training data

In [11]:
ts = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# Select the foundation model you want to customize (you can find this from the "modelId" from listed foundation model above)
base_model_id = "amazon.titan-image-generator-v1:0"

# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING". 
customization_type = "FINE_TUNING"

# Specify the roleArn for your customization job
customization_role = role_arn

# Create a customization job name
customization_job_name = f"image-gen-ft-{ts}"

# Create a customized model name for your fine-tuned Llama2 model
custom_model_name = f"image-gen-ft-{ts}"

# Define the hyperparameters for fine-tuning Llama2 model
hyper_parameters = {
    "stepCount": "8000",
    "batchSize": "8",
    "learningRate": "0.00001",
}

# Specify your data path for training, validation(optional) and output
s3_train_uri = s3_bucket_path + "/" + output_file
training_data_config = {"s3Uri": s3_train_uri}


output_data_config = {"s3Uri": f's3://{bucket_name}/outputs/output-{custom_model_name}'}

# Create the customization job
bedrock_client.create_model_customization_job(
    customizationType=customization_type,
    jobName=customization_job_name,
    customModelName=custom_model_name,
    roleArn=customization_role,
    baseModelIdentifier=base_model_id,
    hyperParameters=hyper_parameters,
    trainingDataConfig=training_data_config,
    outputDataConfig=output_data_config
)


{'ResponseMetadata': {'RequestId': 'c1c09ec1-3658-4593-b986-e13cb80ae3e9',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 13:43:03 GMT',
   'content-type': 'application/json',
   'content-length': '122',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c1c09ec1-3658-4593-b986-e13cb80ae3e9'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-east-1:094784590684:model-customization-job/amazon.titan-image-generator-v1:0/hw5vebkzf8c1'}

### Waiting until customization job is completed
Once the customization job is finished, you can check your existing custom model(s) and retrieve the modelArn of your fine-tuned model.

<div class=\"alert alert-block alert-warning\">
    <b>Warning:</b> The model customization job can take hours to run. With 5000 steps, 0.000001 learning rate, 64 of batch size and 60 images, it takes around 4 hours to complete
</div>


In [ ]:

# check model customization status
status = bedrock_client.list_model_customization_jobs(
    nameContains=customization_job_name
)["modelCustomizationJobSummaries"][0]["status"]
while status == 'InProgress':
    time.sleep(50)
    status = bedrock_client.list_model_customization_jobs(
        nameContains=customization_job_name
    )["modelCustomizationJobSummaries"][0]["status"]
    print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress

## Next Steps

Once your training job is completed, you can run the `TitanImageGenerator - Create PT and Invoke Model` notebook to invoke the model

In [13]:
%store customization_role

Stored 'customization_role' (str)
